# Movie Sentiment Analysis with Keras

In [29]:
# uncomment these for Google collab, will have already been installed in local environment 
# if 'pip install -r requirements.txt' has been run
#!pip install nltk
#!pip install --upgrade gensim

import numpy as np
import os
import os.path

from pdb import set_trace
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
import nltk


import glob
from gensim.models import Word2Vec

import time

[nltk_data] Downloading package punkt to /home/jeremie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
# MacOSX: See https://www.mkyong.com/mac/wget-on-mac-os-x/ for wget
if not os.path.isdir('../aclImdb'):
    if not os.path.isfile('../aclImdb_v1.tar.gz'):
      !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 

    if not os.path.isdir('../aclImdb'):  
      !tar -xf aclImdb_v1.tar.gz 

In [31]:
time_beginning_of_notebook = time.time()
SAMPLE_SIZE=3000
positive_sample_file_list = glob.glob(os.path.join('../aclImdb/train/pos', "*.txt"))
positive_sample_file_list = positive_sample_file_list[:SAMPLE_SIZE]

negative_sample_file_list = glob.glob(os.path.join('../aclImdb/train/neg', "*.txt"))
negative_sample_file_list = negative_sample_file_list[:SAMPLE_SIZE]

import re

# load doc into memory
# regex to clean markup elements 
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r', encoding='utf8')
    # read all text
    text = re.sub('<[^>]*>', ' ', file.read())
    #text = file.read()
    # close the file
    file.close()
    return text


In [32]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

df_positives = pd.DataFrame({'reviews':[load_doc(x) for x in positive_sample_file_list], 'sentiment': np.ones(SAMPLE_SIZE)})
df_negatives = pd.DataFrame({'reviews':[load_doc(x) for x in negative_sample_file_list], 'sentiment': np.zeros(SAMPLE_SIZE)})

print("Positive review(s):", df_positives['reviews'][1][:100])
print("Negative review(s):", df_negatives['reviews'][1][:100])

df = pd.concat([df_positives, df_negatives], ignore_index=True)

df = shuffle(df)

X_train, X_test, y_train, y_test = train_test_split(df['reviews'], df['sentiment'], test_size=0.25)


Positive review(s): NYC model Alison Parker (Cristina Raines) rents a room in an old brownstone where she meets a few bi
Negative review(s): The reason the DVD releases of this film are in black and white is because nobody can get their hand


In [60]:
import tensorflow as tf

# def lstm_keras():
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM, Dropout
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing import sequence

In [51]:


vocab_size = 1000

# Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',\
#           lower=True, split=' ', char_level=False, oov_token=None, document_count=0)

tokenize = Tokenizer(num_words=vocab_size)
tokenize.fit_on_texts(X_train)

tokenized_X_train = tokenize.texts_to_sequences(X_train)
tokenized_X_test = tokenize.texts_to_sequences(X_test)

max_document_length = max([len(x) for x in np.concatenate((tokenized_X_train,tokenized_X_test),axis=0)])
encoded_X_train = sequence.pad_sequences(tokenized_X_train, maxlen=max_document_length)
encoded_X_test = sequence.pad_sequences(tokenized_X_test, maxlen=max_document_length)


encoder = LabelBinarizer()
encoder.fit(y_train)
encoded_y_train = encoder.transform(y_train)
encoded_y_test = encoder.transform(y_test)

In [ ]:
https://stackoverflow.com/questions/51031519/cant-import-keras-layers-merge
https://github.com/keras-team/keras/issues/3921
https://github.com/keras-team/keras/issues/6547

In [63]:
from keras.layers import Concatenate

# create the model
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_document_length))

branches = [] # models to be merged
for filter_len in [3,4,5]:
    branch = Sequential()
    branch.add(Conv1D(filters=128, kernel_size=filter_len, padding='same', activation='relu'))
    branch.add(MaxPooling1D())
    branch.add(Flatten())
    branches.append(branch)    
model.add(Concatenate(branches))

model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

adam = Adam()
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy','categorical_accuracy'])

model.summary()

ValueError: A `Concatenate` layer should be called on a list of at least 2 inputs

In [ ]:


model.fit(reviews,labels,batch_size=batch_size,epochs=num_epochs,\
          validation_data=(reviews_val,labels_val))